In [2]:
## From SQL to DataFrame Pandas
## Ref: https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e

import pandas as pd
import pyodbc
import numpy as np

sql_conn = pyodbc.connect(DRIVER='{ODBC Driver 13 for SQL Server}'
                           , SERVER='*Server Name*'
                           , DATABASE='*dbName*'
                            , Trusted_Connection='yes') 
query = "SELECT Top 5000 [Engagement Name], [DataSource],[Calendar Date],[Sales Location Name],[RegularHours],[ActualTransactionAmtCUS] FROM CPWorkedHoursFactCurrent CP JOIN vwFinanceGeographyDimension FG ON FG.[Finance Geo Original Key]=CP.FinanceGeoKey JOIN vwDateDimension DD ON DD.[Date Key]=cp.TransactionDateKey JOIN vwMCSEngagementDimension ED ON ED.[Engagement Key]=CP.[EngagementKey] WHERE DataSource IN ( 'Changepoint') UNION  SELECT Top 5000 [Engagement Name], [DataSource],[Calendar Date],[Sales Location Name],[RegularHours],[ActualTransactionAmtCUS] FROM CPWorkedHoursFactCurrent CP JOIN  vwFinanceGeographyDimension FG ON FG.[Finance Geo Original Key]=CP.FinanceGeoKey JOIN vwDateDimension DD ON DD.[Date Key]=cp.TransactionDateKey  JOIN vwMCSEngagementDimension ED ON ED.[Engagement Key]=CP.[EngagementKey] WHERE DataSource IN ('SAP')"
eng_details = pd.read_sql(query, sql_conn)

eng_details.head(10)

,Engagement Name,DataSource,Calendar Date,Sales Location Name,RegularHours,ActualTransactionAmtCUS
0,FF-2083818-NSG-NIP-120421-137246-INSCOM DAS-Ba...,Changepoint,2018-04-13,United States,8.0,NaN
1,PRBL-TM-COMPAREX-MCS BANKINTER-S4B Large Meeti...,Changepoint,2018-02-19,Spain,4.0,NaN
2,CN15E063-医利捷(上海)信息科技有限公司-上海市同仁医院-T&M Project,Changepoint,2017-10-16,UNKNOWN,8.0,NaN
3,CN15E063-医利捷(上海)信息科技有限公司-上海市同仁医院-T&M Project,Changepoint,2017-09-04,UNKNOWN,10.0,NaN
4,FF-TOO Agile Solutions-Atyrau City Adm Smart C...,Changepoint,2018-01-12,Kazakhstan,8.0,NaN
5,OM-MCS-16DEC20-ESWO-IITC-PDORoadmap-TM\t,Changepoint,2017-10-19,Oman,3.0,937.5
6,ZRT_HYBRID-PROJECT AURORA,SAP,2018-08-31,UNKNOWN,NaN,36000.0
7,SVSPRE-SWD-EchoStar-Renew-FY17,SAP,2018-09-26,United States,5.0,5000.0
8,PRBL-Computa-MINDEF-WIN10-ASSIST-17-01,Changepoint,2017-07-20,France,8.0,NaN
9,PRBL-Computa-MINDEF-WIN10-ASSIST-17-01,Changepoint,2017-07-03,France,4.0,NaN


In [10]:
eng_details.groupby ("Sales Location Name").mean().head()

,RegularHours,ActualTransactionAmtCUS
Sales Location Name,,
Albania,3.560241,17847.600000
Bulgaria,2.833333,27490.400000
Canada,NaN,3318.797784
Chile,NaN,7072.201613
China,7.488372,26235.000000


In [15]:
#Below, we group on more than one field. Pandas will sort things on the same list of fields by default, so there’s no need for a .sort_values() in the first example
eng_details.groupby ("Sales Location Name").size().head()

Sales Location Name
Albania     90
Bulgaria     8
Canada      19
Chile       22
China       45
dtype: int64

In [32]:

eng_details.groupby (['Sales Location Name' , 'DataSource']).size().to_frame('size').reset_index().sort_values (['Sales Location Name','size'], ascending =[False, False]).head()


,Sales Location Name,DataSource,size
28,United States,Changepoint,938
29,United States,SAP,55
27,United Kingdom,Changepoint,2
25,UNKNOWN,Changepoint,993
26,UNKNOWN,SAP,659
